This notebook is to understand how runnables work internally in langchain. In this notebook I will create dummy llm, template, runnable connector and dummy parser. Then I will create chains and connect them using Runnable connector. Finally I will invoke chain by just one invoke.

In [1]:
from abc import ABC, abstractmethod
import random

In [2]:
class Runnable(ABC):
    @abstractmethod
    def invoke(input_data):
        pass

In [3]:
class dummyLLM(Runnable):
    def __init__(self):
        print('LLM created')
    
    def predict(self, prompt):
        response_list = [
            'Rome is capital of Italy', 
            'Polito is best university', 
            'AI stands for Artifical Intelligence'
        ]
        return {'response': random.choice(response_list) }
    
    def invoke(self, input_data):
        response_list = [
            'Rome is capital of Italy', 
            'Polito is best university', 
            'AI stands for Artifical Intelligence'
        ]
        return {'response': random.choice(response_list) }

In [4]:
class dummyPromptTemplate(Runnable):
    def __init__(self, template, input_variables):
        self.template = template
        self.input_variables = input_variables

    def invoke(self,input_dict):
        return self.template.format(**input_dict)
    
    def format(self, input_dict):
        return self.template.format(**input_dict)

In [5]:
class dummyStrOutputParser(Runnable):
    def __init__(self):
        pass

    def invoke(self,input_data):
        return input_data['response'] 

In [6]:
class RunnableConnector(Runnable):
    def __init__(self, runnable_list):
        self.runnable_list = runnable_list

    def invoke(self, input_data):
        for runnable in self.runnable_list:
            input_data = runnable.invoke(input_data)
        return input_data

In [7]:
template = dummyPromptTemplate(
    template= 'write a {length} poem about {topic}', 
    input_variables= ['length', 'topic']
)

In [8]:
llm = dummyLLM()

LLM created


In [9]:
parser = dummyStrOutputParser()

In [10]:
chain = RunnableConnector([template, llm, parser])

In [12]:
chain.invoke({'length': 'short', 'topic': 'italy'})

'Polito is best university'

In [13]:
template1 = dummyPromptTemplate(
    template= 'write a joke about {topic}', 
    input_variables= ['topic']
)

In [14]:
template2 = dummyPromptTemplate(
    template= 'explain the following joke {response}', 
    input_variables= ['response']
)

In [15]:
llm = dummyLLM()

LLM created


In [16]:
parser = dummyStrOutputParser()

In [18]:
chain1 = RunnableConnector([template1, llm])

In [20]:
chain2 = RunnableConnector([template2, llm, parser])

In [21]:
final_chain = RunnableConnector([chain1, chain2])

In [22]:
final_chain.invoke({'topic': 'cricket'})

'Polito is best university'